### Import data

In [2]:
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
#from google.colab import drive
from numpy import cumsum
from os import walk
import pandas as pd 
import numpy as np
import math

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, DepthwiseConv2D
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from keras import backend as K

#drive.mount('/content/gdrive')

In [3]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def abundant(df):    
    second = 3

    l = df['second']
    l = np.array(l)
    l_check = np.append(l[1:],l[-1])
    l_diff = l_check - l
    l_ind = l_diff <= second  
    df = df[l_ind]

    return df

def min_max(lst,mn,mx):
    lst = np.array(lst)
    lst[lst < mn] = mn
    lst[lst > mx] = mx
    return lst  

def preprocessing(data,maximum_accuracy,low_speed):
  
    # Clean high accuracy and low_speed
    data = data[data['Accuracy'] <= maximum_accuracy]
    data = data[data['Speed'] >= low_speed]

    # Clean abundant data
    data = data.groupby('bookingID').apply(abundant)     

    # Clean outliner
    data['acceleration_x'] = min_max(data['acceleration_x'],-3,3) / 3
    data['acceleration_y'] = min_max(data['acceleration_y'],-10,10) / 10
    data['acceleration_z'] = min_max(data['acceleration_z'],-5,5) / 5
    data['gyro_x'] = min_max(data['gyro_x'],-5,5) / 5
    data['gyro_y'] = min_max(data['gyro_x'],-0.4,0.4) / 0.4
    data['gyro_z'] = min_max(data['gyro_x'],-0.3,0.3) / 0.3 
    data['Speed'] = min_max(data['Speed'],0,100) / 100 

    # Remove columns
    data = data[['bookingID','acceleration_x', 'acceleration_y','acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z','Speed']]  

    # Total acc , gyro
    data['total_acceleration'] = np.sqrt(np.power(data['acceleration_x'], 2) + np.power(data['acceleration_y'], 2) + np.power(data['acceleration_z'], 2))
    data['total_gyro'] = np.sqrt(np.power(data['gyro_x'], 2) + np.power(data['gyro_y'], 2) + np.power(data['gyro_z'], 2))  

    #Change format 
    col = data.columns
    data = pd.DataFrame(data.values,columns= col)

    return data 

window_size = 120
def slice_(df,w = window_size):
    nrow = df.shape[0]
    lenght = math.floor(nrow/w) * w
    df = df.head(lenght) 
    return df

def frame(df,window_size):
    col = df.columns
    df = df.groupby('bookingID').apply(slice_)
    df = pd.DataFrame(df.values,columns= col)
    df = df[['acceleration_x','acceleration_y','acceleration_z','gyro_x','gyro_y','gyro_z','Speed','total_acceleration','total_gyro']]

    n = int(df.shape[0] / window_size)
    df = df.values.reshape(n,window_size,3,3)
    return df

### Load data

In [4]:
mypath = '/content/gdrive/My Drive/Grab/features/'

dataset = pd.DataFrame()
for (dirpath, dirnames, filenames) in walk(mypath):    
    for file in filenames:
        if not file.startswith('.'):
            print(file)
            f = mypath + file
            df = pd.read_csv(f)
            dataset = pd.concat([dataset,df])

labels = pd.read_csv("/content/gdrive/My Drive/Grab/labels.csv")
dataset = dataset.sort_values(by=['bookingID','second'])        
dataset['Speed'] = dataset['Speed'] * 18/5

part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv


In [0]:
df_cln = preprocessing(dataset,maximum_accuracy = 30, low_speed= 10)

In [0]:
window_size = 120
s = df_cln.groupby('bookingID').size()
s = s.reset_index()
s.columns = ['bookingID','count']
s = s.loc[s['count']>=window_size]
s = list(s['bookingID'])

df_cln = df_cln.merge(labels,on = 'bookingID')
df_cln = df_cln[df_cln['bookingID'].isin(s)]

#### Splitting Frame

In [9]:
df_cln_1 = df_cln[df_cln['label'] == 1]
df_cln_0 = df_cln[df_cln['label'] == 0]

df_cln_1 = frame(df_cln_1,window_size)
df_cln_0 = frame(df_cln_0,window_size)
print("labels 1 :",df_cln_1.shape)
print("labels 0 :",df_cln_0.shape)

labels 1 : (23226, 120, 3, 3)
labels 0 : (54634, 120, 3, 3)


In [0]:
no_1 = len(df_cln_1)
no_0 = len(df_cln_0)
labels = pd.Series(np.append(np.repeat(1, no_1),np.repeat(0, no_0)))
label = np.asarray(pd.get_dummies(labels), dtype = np.int8)

In [0]:
X = np.append(df_cln_1,df_cln_0,axis = 0)
X.shape

del df_cln_1,df_cln_0

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2 , random_state=456)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print("X_train: ",X_train.shape)
print('X_test: ',X_test.shape)

print("y_train: ",y_train.shape)
print('y_test: ',y_test.shape)

X_train:  (62288, 120, 3, 3)
X_test:  (15572, 120, 3, 3)
y_train:  (62288, 2)
y_test:  (15572, 2)


In [0]:
window_size =120
batch_size = 128
input_shape = (window_size,3,3)
output = 2
learning_rate = 0.005
epochs = 150

#### Deptwise

In [67]:
model = Sequential()
model.add(DepthwiseConv2D(kernel_size = (10,3), activation = 'relu', padding = 'same',input_shape=input_shape))
model.add(DepthwiseConv2D(kernel_size = (5,3), activation = 'relu', padding = 'same',input_shape=input_shape))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(output, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_1 (Depthwis (None, 120, 3, 3)         93        
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (None, 120, 3, 3)         48        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               138368    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 138,767
Trainable params: 138,767
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = keras.optimizers.rmsprop(lr=learning_rate, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=[auc])

In [70]:
model.fit(X_train, y_train,batch_size = batch_size, epochs=epochs,validation_data=(X_test, y_test),shuffle=True,verbose=1)

Train on 62288 samples, validate on 15572 samples
Epoch 1/150
62288/62288 [==============================] - 5s 82us/step - loss: 0.6124 - auc: 0.7051 - val_loss: 0.6095 - val_auc: 0.7093
Epoch 2/150
62288/62288 [==============================] - 5s 76us/step - loss: 0.6058 - auc: 0.7098 - val_loss: 0.6081 - val_auc: 0.7107
Epoch 3/150
62288/62288 [==============================] - 5s 76us/step - loss: 0.6034 - auc: 0.7119 - val_loss: 0.6057 - val_auc: 0.7124
Epoch 4/150
62288/62288 [==============================] - 5s 76us/step - loss: 0.6014 - auc: 0.7136 - val_loss: 0.6079 - val_auc: 0.7144
Epoch 5/150
62288/62288 [==============================] - 5s 76us/step - loss: 0.5999 - auc: 0.7155 - val_loss: 0.6105 - val_auc: 0.7163
Epoch 6/150
62288/62288 [==============================] - 5s 75us/step - loss: 0.5981 - auc: 0.7173 - val_loss: 0.6133 - val_auc: 0.7181
Epoch 7/150
62288/62288 [==============================] - 5s 75us/step - loss: 0.5966 - auc: 0.7192 - val_loss: 0.6136 - 

In [71]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test AUC:', scores[1])

15572/15572 [==============================] - 1s 67us/step
Test loss: 2.7495952866360978
Test AUC: 0.8741542684398027


In [0]:
model.save('/content/gdrive/My Drive/Grab/my_model3.h5')

In [0]:
#keras.backend.clear_session()

#### Test

In [ ]:
Testset = pd.read_csv("test.csv")

In [5]:
model = load_model('my_model3.h5', custom_objects={'auc': auc})

In [7]:
def pred_test(dataset,window_size = 120,thres = 0.5):
    
    id_f = np.unique(dataset['bookingID'])
    dataset = preprocessing(dataset,30,10) 

    d = dataset.groupby('bookingID').size()
    d = d.reset_index()
    d.columns = ['bookingID','count']
    s = list(d['bookingID'][d['count'] >= window_size])

    id_lenght = d[d['count'] >= window_size]
    id_lenght['n_frame'] = np.floor(d['count']/ window_size)

    df = dataset[dataset['bookingID'].isin(s)]
    X = frame(df,window_size)
    m_pred = model.predict_classes(X)

    ID = np.array(id_lenght['bookingID']).astype(int)
    N = np.array(id_lenght['n_frame']).astype(int)
    ID_N = np.array([])
    for i,j in zip(ID,N):
        ID_N = np.append(ID_N,np.repeat(i,j))

    pred = pd.DataFrame({'bookingID' :ID_N,'pred' : m_pred}) 
    pred = pred.groupby('bookingID')['pred'].mean()
    pred = pd.DataFrame(pred)
    pred = pred.reset_index()
    pred.loc[pred['pred']  > thres,'pred'] = 1
    pred.loc[pred['pred'] <= thres,'pred'] = 0

    id_f = pd.DataFrame({'bookingID':id_f})
    id_f = id_f.merge(pred,how ='left', on = 'bookingID')
    id_f = id_f.fillna(0)
    return(id_f)

In [9]:
id_f = pred_test(dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [0]:
labels = pd.read_csv("labels.csv")

In [11]:
id_f = id_f.merge(labels,on = 'bookingID')
roc_auc_score(id_f['pred'],id_f['label'])

0.9170124738875937